In [1]:
import os 
import pandas as pd
import numpy as np

directory = os.getcwd()

cwd = directory.replace('Strat_7', '\module_1')
os.chdir(cwd)

import Preprocessing_functions as pf 
import calculateMaxDD 
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import joblib
os.getcwd()

'c:\\Users\\User\\Documents\\ATS_Development\\module_1'

In [2]:
ticker = 'QQQ'

df = pf.downlaod_symbol_data(ticker, period= '120mo')
try:
    df = df.drop(columns=['Stock Splits', 'Dividends', 'Capital Gains'])
except KeyError:
    print("Columns not available (see above line of code)")
    
df = pf.create_momentum_feat(df, symbol=ticker) ### need to inspect in more detail how the create momemntum features work and the shift in this case
df = pf.technical_indicators(df,MA_DIVERGENCE=True)
df = pf.format_idx_date(df)

#df['prev_close'] = df['Close'].shift(1)
#df['tom_open'] = df['Open'].shift(-1)

df['overnight_pct'] = (df['Open'].shift(-1) - df['Close']) / df['Close']
#df['overnight_pct'] = (df['Open'] - df['Close'].shift(1)) / df['Close'].shift(1) WRONG - OVERFITTING!

df = df[df.index <= '2024-02-01']

df = df.dropna()

df.tail()

,Open,High,Low,Close,Volume,open_low,open_close,open_high,high_low,low_close,...,MACD,diff_Close_8,diff_Close_25,diff_Close_50,diff_Close_100,diff_Close_200,diff_8_50,diff_50_100,diff_50_200,overnight_pct
Date,,,,,,,,,,,,,,,,,,,,,
2024-01-26,421.93,423.69,420.65,421.31,37137000,0.303368,0.146944,-0.417131,0.717506,-0.156900,...,1.132469,3.47125,13.1064,22.5506,41.3632,56.42015,19.07935,18.8126,33.86955,0.001068
2024-01-29,421.76,425.82,421.11,425.62,38694700,0.154116,-0.915212,-0.962633,1.106101,-1.070979,...,1.134347,5.17875,16.4112,25.9900,45.1605,60.14970,20.81125,19.1705,34.15970,-0.002443
2024-01-30,424.58,425.15,421.87,422.79,36739000,0.638278,0.421593,-0.134250,0.771492,-0.218077,...,0.838687,0.81875,12.8756,22.3518,41.8131,56.78375,21.53305,19.4613,34.43195,-0.010478
2024-01-31,418.36,420.37,414.33,414.51,64010600,0.963285,0.920260,-0.480447,1.436829,-0.043444,...,0.022316,-6.93875,4.2452,13.4356,33.0720,48.00620,20.37435,19.6364,34.57060,0.004367
2024-02-01,416.32,419.88,415.34,419.39,51008600,0.235396,-0.737414,-0.855111,1.081261,-0.975105,...,-0.237841,-2.07750,8.6792,17.5836,37.4472,52.36560,19.66110,19.8636,34.78200,0.004554


In [6]:
pf.add_market_feature('^VIX', time_period='120mo', data=df)


ModuleNotFoundError: No module named 'ALGO_KT1'

In [ ]:
print('Start date of dataframe: ', df.index.min())
print('End date of dataframe: ', df.index.max())

In [ ]:
y = df.pop('overnight_pct')
X = df.copy()
X = X.drop(columns= ['Open', 'High', 'Low', 'Close',])

# Split data into train (60%), validation (20%), and test (20%) sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


In [ ]:
X.columns

In [ ]:
model_columns = pd.DataFrame(X.columns, columns=['features'])

model_columns.to_csv(directory + f'/model_features/{ticker}_features_{len(X.columns)}.csv', index = False)


In [ ]:
# Define Random Forest model
rf = RandomForestRegressor(random_state=42)

# Grid Search for best hyperparameters
param_grid = {
    "n_estimators": [50, 100, 200],
    "max_depth": [None, 10, 20],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4]
}

grid_search = GridSearchCV(rf, param_grid, cv=10, scoring='neg_mean_squared_error', n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

# Best parameters from Grid Search
best_rf = grid_search.best_estimator_
print("Best Parameters:", grid_search.best_params_)

In [ ]:
# Cross-validation with 10-folds
cv_scores = cross_val_score(best_rf, X_train, y_train, cv=10, scoring='neg_mean_squared_error')
rmse_cv = np.sqrt(-cv_scores)
print(f"10-Fold CV RMSE: {rmse_cv.mean():.4f} ± {rmse_cv.std():.4f}")

In [ ]:
# Evaluate on validation set
y_val_pred = best_rf.predict(X_val)
rmse_val = np.sqrt(mean_squared_error(y_val, y_val_pred))
print(f"Validation RMSE: {rmse_val:.4f}")

In [ ]:
X_val.head()

In [ ]:
# Final evaluation on test set
y_test_pred = best_rf.predict(X_test)
rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
print(f"Test RMSE: {rmse_test:.4f}")

In [ ]:
os.chdir(cwd.replace('module_1', 'Strat_7\models'))

In [ ]:
os.getcwd()

In [ ]:
# Save the best model
os.chdir(cwd.replace('module_1', 'Strat_7\models'))
joblib.dump(best_rf, f"{ticker}_overnight_regression_random_forest.pkl")
print(f"Model saved as {ticker}_overnight_regression_random_forest.pkl")



# Save the best model
joblib.dump(best_rf, "best_random_forest.pkl")
print("Model saved as best_random_forest.pkl")

# Load the model for future use
loaded_model = joblib.load("best_random_forest.pkl")
y_loaded_pred = loaded_model.predict(X_test)

#### BACKTESTING

In [ ]:
#ticker = 'SPY'

df1 = pf.downlaod_symbol_data(ticker, period= '360mo')
try:
    df1 = df1.drop(columns=['Stock Splits', 'Dividends', 'Capital Gains'])
except KeyError:
    print("Columns not available (see above line of code)")
    
df1 = pf.create_momentum_feat(df1, symbol=ticker) ### need to inspect in more detail how the create momemntum features work and the shift in this case
df1 = pf.technical_indicators(df1,MA_DIVERGENCE=True)
df1 = pf.format_idx_date(df1)

df1 = df1.dropna()

model_start_date = df.index.min()

df1 = df1[df1.index < model_start_date]

df1.head()

In [ ]:
X.columns

In [ ]:
df2 = df1[X.columns]

df2.head()

In [ ]:
os.chdir(cwd.replace('module_1', 'Strat_7\models'))
os.getcwd()

In [ ]:
best_rf = joblib.load(f"{ticker}_overnight_regression_random_forest.pkl")

In [ ]:
df1['predictions'] = best_rf.predict(df2)
df1['overnight_pct'] = (df1['Open'].shift(-1) - df1['Close']) / df1['Close']
df1['action'] = np.where(df1['predictions'] > 0, 'BUY', 'SELL')
df1.head()

In [ ]:
df1 = df1.dropna()
df1['prev_close'] = df1['Close'].shift()
df1['tom_open'] = df1['Open'].shift(-1)

cols = df1.columns
cols = [i for i in cols if i not in X.columns]

df1 = df1[cols]

df1.head()

In [ ]:
import matplotlib.pyplot as plt 

df1['error'] = df1['predictions'] - df1['overnight_pct']

plt.figure(figsize=[10,7])
plt.plot(df1.index, df1['error'] , color = 'b')
plt.title('Overning Error in PCT points')
plt.xlabel('Date')
plt.ylabel('Pct Difference')

In [ ]:
df1.head()

In [ ]:
df1['return'] = np.where(df1['action'] == 'SELL', df1['overnight_pct'] * (-1), df1['overnight_pct'])
df1.head()

In [ ]:
df1['cum_ret'] = df1['return'].cumsum()

plt.figure(figsize=[10,7])
plt.plot(df1.index, df1['cum_ret'] , color = 'b')
plt.title('Backtest Cummulative Return PCT')
plt.xlabel('Date')
plt.ylabel('Cummulative Return PCT')

In [ ]:
##### LOSSES
df1[df1['return'] < 0]

In [ ]:

#####   MAX DRAWDOWN
#from calculateMaxDD import calculateMaxDD

cum_ret = np.cumprod(1+ df1['return']) - 1
maxDrawdown, maxDrawdownDuration, startDrawdownDay=calculateMaxDD.calculateMaxDD(cum_ret.values)

#####   SHARPE RATIO
sharpe_ratio = round(np.sqrt(252) * np.mean(df1['return']) / np.std(df1['return']),2)

#####   AVG YEARLY RETURN AND STD
mean_ret = df1['return'].mean() * 252
std = df1['return'].std()*np.sqrt(252)

print(f'Sharpe Ratio: {sharpe_ratio}')
print(f'Maximum Drawdown: {round(maxDrawdown,4)}')
print(f'Max Drawdown Duration: {maxDrawdownDuration} days' )
print(f'Start day Drawdown: {startDrawdownDay}')
print(f"Average Yearly Return: {round(mean_ret*100, 2)} %")

In [ ]:
rets = df1[['return']]
rets.to_csv(directory + '\strat_returns' + f'\{ticker}.csv')

In [ ]:
directory

In [ ]:
df1.predictions.describe()